In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from ultralytics import YOLO
import yaml

# Add src to path
sys.path.insert(0, '../')
from src.utils.visualization import draw_boxes, visualize_detection

plt.rcParams['figure.figsize'] = (15, 10)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 1. Load Trained Model

In [ ]:
# Path to trained model
model_path = '../models/best.pt'  # Update this path

# Check if model exists
if Path(model_path).exists():
    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)
    print("Model loaded successfully!")
    
    # Print model info
    print(f"\nModel Summary:")
    print(f"  Task: {model.task}")
    print(f"  Device: {model.device}")
else:
    print(f"⚠️  Model not found at {model_path}")
    print("Please train a model first using: python scripts/train.py")
    print("\nFor testing purposes, we'll load a pretrained YOLOv8 model:")
    model = YOLO('yolov8n.pt')
    print("Loaded pretrained YOLOv8n model")

## 2. Load Dataset Configuration

In [ ]:
# Load dataset configuration for class names
config_path = '../configs/dataset.yaml'

with open(config_path, 'r') as f:
    dataset_config = yaml.safe_load(f)

class_names = dataset_config['names']
print(f"Class names: {class_names}")

## 3. Single Image Inference

In [ ]:
def run_inference(image_path, conf_threshold=0.25, iou_threshold=0.45):
    """Run inference on a single image."""
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model.predict(
        source=image_path,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=False
    )[0]
    
    # Extract predictions
    boxes = results.boxes.xyxy.cpu().numpy()
    scores = results.boxes.conf.cpu().numpy()
    labels = results.boxes.cls.cpu().numpy().astype(int)
    
    # Draw boxes
    if len(boxes) > 0:
        image_rgb = draw_boxes(
            image_rgb,
            boxes,
            labels.tolist(),
            scores,
            class_names
        )
    
    # Display
    plt.figure(figsize=(15, 10))
    plt.imshow(image_rgb)
    plt.title(f'Detections: {len(boxes)} objects found')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Print details
    print(f"\nImage: {Path(image_path).name}")
    print(f"Detections: {len(boxes)}")
    
    if len(boxes) > 0:
        print("\nDetailed Results:")
        for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
            class_name = class_names[label] if label in class_names else f"Class {label}"
            print(f"  {i+1}. {class_name}: {score:.3f} - [{box[0]:.1f}, {box[1]:.1f}, {box[2]:.1f}, {box[3]:.1f}]")
    
    return boxes, scores, labels

# Example: Test on a sample image
# Replace with your own image path
test_image = '../data/images/test/sample.jpg'  # Update this path

if Path(test_image).exists():
    results = run_inference(test_image, conf_threshold=0.25)
else:
    print(f"Test image not found: {test_image}")
    print("Please provide a valid image path")

## 4. Batch Inference on Directory

In [ ]:
def batch_inference(image_dir, conf_threshold=0.25, max_display=6):
    """Run inference on all images in a directory."""
    image_dir = Path(image_dir)
    
    # Find all images
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = []
    for ext in image_extensions:
        image_files.extend(image_dir.glob(f"*{ext}"))
    
    if not image_files:
        print(f"No images found in {image_dir}")
        return
    
    print(f"Found {len(image_files)} images")
    
    # Display first N results
    n_display = min(max_display, len(image_files))
    n_cols = 3
    n_rows = (n_display + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6*n_rows))
    axes = axes.flatten() if n_display > 1 else [axes]
    
    all_detections = []
    
    for i, img_path in enumerate(image_files):
        # Run inference
        image = cv2.imread(str(img_path))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        results = model.predict(
            source=str(img_path),
            conf=conf_threshold,
            verbose=False
        )[0]
        
        boxes = results.boxes.xyxy.cpu().numpy()
        scores = results.boxes.conf.cpu().numpy()
        labels = results.boxes.cls.cpu().numpy().astype(int)
        
        all_detections.append(len(boxes))
        
        # Draw and display
        if i < n_display:
            if len(boxes) > 0:
                image_rgb = draw_boxes(image_rgb, boxes, labels.tolist(), scores, class_names)
            
            axes[i].imshow(image_rgb)
            axes[i].set_title(f"{img_path.name}\n{len(boxes)} detections")
            axes[i].axis('off')
    
    # Hide extra subplots
    for i in range(n_display, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\nBatch Inference Summary:")
    print(f"  Total images: {len(image_files)}")
    print(f"  Total detections: {sum(all_detections)}")
    print(f"  Average detections per image: {np.mean(all_detections):.2f}")
    print(f"  Images with detections: {sum(1 for d in all_detections if d > 0)}")

# Example: Run batch inference
test_dir = '../data/images/test'  # Update this path

if Path(test_dir).exists():
    batch_inference(test_dir, conf_threshold=0.25, max_display=6)
else:
    print(f"Test directory not found: {test_dir}")

## 5. Adjust Detection Thresholds

In [ ]:
def compare_thresholds(image_path, thresholds=[0.1, 0.25, 0.5, 0.75]):
    """Compare detection results at different confidence thresholds."""
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, threshold in enumerate(thresholds):
        # Run inference
        results = model.predict(
            source=image_path,
            conf=threshold,
            verbose=False
        )[0]
        
        boxes = results.boxes.xyxy.cpu().numpy()
        scores = results.boxes.conf.cpu().numpy()
        labels = results.boxes.cls.cpu().numpy().astype(int)
        
        # Draw boxes
        img_display = image_rgb.copy()
        if len(boxes) > 0:
            img_display = draw_boxes(img_display, boxes, labels.tolist(), scores, class_names)
        
        axes[idx].imshow(img_display)
        axes[idx].set_title(f"Confidence threshold: {threshold}\n{len(boxes)} detections")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Example: Compare thresholds
if Path(test_image).exists():
    compare_thresholds(test_image, thresholds=[0.1, 0.25, 0.5, 0.75])
else:
    print("Please provide a valid test image")

## 6. Model Performance Analysis

In [ ]:
# If you have a trained model, you can analyze its performance
if Path(model_path).exists():
    print("Model Performance Metrics:")
    print("(Run evaluation script for detailed metrics)")
    print("\nTo evaluate your model, run:")
    print("  python scripts/evaluate.py --model models/best.pt --data configs/dataset.yaml")
else:
    print("No trained model available yet.")
    print("Train a model first using:")
    print("  python scripts/train.py --data configs/dataset.yaml --epochs 100")

## 7. Export Model for Deployment

In [ ]:
def export_model(model_path, format='onnx'):
    """Export model to different formats for deployment."""
    if not Path(model_path).exists():
        print(f"Model not found: {model_path}")
        return
    
    model = YOLO(model_path)
    
    print(f"Exporting model to {format.upper()} format...")
    
    try:
        export_path = model.export(format=format)
        print(f"✓ Model exported successfully to: {export_path}")
    except Exception as e:
        print(f"✗ Export failed: {e}")

# Example: Export to ONNX format
if Path(model_path).exists():
    print("\nAvailable export formats:")
    print("  - onnx: Cross-platform deployment")
    print("  - torchscript: PyTorch deployment")
    print("  - coreml: iOS/macOS deployment")
    print("  - engine: TensorRT for NVIDIA GPUs")
    print("  - tflite: TensorFlow Lite for mobile")
    print("\nTo export, uncomment and run:")
    print("  # export_model(model_path, format='onnx')")
else:
    print("Train a model first before exporting")

## Next Steps

1. **Collect More Data**: Improve model by adding more diverse graffiti examples
2. **Fine-tune**: Adjust hyperparameters based on performance
3. **Deploy**: Export model and integrate into your application
4. **Monitor**: Track real-world performance and retrain as needed